# 필요한 라이브러리 및 모듈

In [1]:
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from gensim.models import doc2vec
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import ast
import numpy as np
import math

def preprocessing(x, stop_words):
    okt = Okt()
    try:
        sentence_tokens = okt.nouns(x)
        #result = ''
        result = []
        for token in sentence_tokens: 
            if token not in stop_words:
                #result += ' ' + token 
                result.append(token)
        return result
    except:
        return ['없음']
        
def rm_whitespace(x):
    result = []
    for i in x:
        if i != '':
            result.append(i)
        else:
            pass
    return result

# evaluation metrices: Precision, Recall, NDCG@K
def compute_metrics(pred_u, target_u, top_k):
    pred_k = pred_u[:top_k]
    num_target_items = len(target_u)

    hits_k = [(i + 1, item) for i, item in enumerate(pred_k) if item in target_u]
    # print("실제로 맞춘 items (position, idx):", hits_k)
    num_hits = len(hits_k)

    idcg_k = 0.0
    for i in range(1, min(num_target_items, top_k) + 1):
        idcg_k += 1 / math.log(i + 1, 2)

    dcg_k = 0.0
    for idx, item in hits_k:
        dcg_k += 1 / math.log(idx + 1, 2)
    
    prec_k = num_hits / top_k
    recall_k = num_hits / min(num_target_items, top_k)
    ndcg_k = dcg_k / idcg_k

    return prec_k, recall_k, ndcg_k

# 조건 설정하는 법
    
    1. 몇 편 이상의 유저로 사용할지
        - 데이터 전처리 파트에 #### code ####와 같이 존재하는 부분으로 이동
        - 아래 코드에 존재하는 숫자가 몇 편 이상으로 할지 선정
            df_user_watch.loc[df_user_watch['cnt_watch']>=10].index
    2. train-test 분리하는 법
        - 데이터 전처리 파트에 마찬가지로 #### code ####와 같이 존재하는 부분으로 이동
        - 랜덤 버전을 할 경우 #랜덤버전 아래의 코드 한줄만 앞의 코드에 #이 없도록 실행

          train, test = train_test_split(df, test_size=0.2, stratify=df['userr'],random_state = 1234)
          #flag_range = int(df.shape[0]*80)
          #df= df.sort_values(by=['year','month','day','hour','min','sec'],ascending=True)
          #train = df[:flag_range]
          #test = df[flag_range:]

        - 기간 선택 버전을 할 경우, #시간 선택 버전 아래의 코드들만 #을 없애고 실행
          
          #train, test = train_test_split(df, test_size=0.2, stratify=df['userr'],random_state = 1234)
          flag_range = int(df.shape[0]*80)
          df= df.sort_values(by=['year','month','day','hour','min','sec'],ascending=True)
          train = df[:flag_range]
          test = df[flag_range:]   
     3. 이전에 시청한 영화를 평가에 포함할지
         - 유저 평가하기 파트에 마찬가지로 #### code ####와 같이 존재하는 부분으로 이동
         - 포함 시킬 경우, 아래와 같이 진행(진행할 코드에만 #을 제거)

            #테스트할때 이전에 시청한 콘텐츠는 제외하고 평가하는것과 포함하여 평가하는것
            # 특정 user가 본 영화들 제외
            #train_items_by_user = train.loc[train.userr==user_id]
            #unique_items = unique_items[~unique_items['iitem1'].isin(train_items_by_user['iitem1'])]
            unique_items = unique_items
            
        - 미포함 시킬 경우, 아래와 같이 진행
        
            #테스트할때 이전에 시청한 콘텐츠는 제외하고 평가하는것과 포함하여 평가하는것
            # 특정 user가 본 영화들 제외
            train_items_by_user = train.loc[train.userr==user_id]
            unique_items = unique_items[~unique_items['iitem1'].isin(train_items_by_user['iitem1'])]
            #unique_items = unique_items
    4. 변수 추가
        - 해당 경우는 워낙 경우의 수가 많아서 완전하게 해드릴 수는 없을 것 같구요. 기본적으로 추가하는 방법을 알려드리지만,
        오류가 나면 저한테 알려주시거나 한번 해결해보세요. 
        - 설명은 적어놓았는데 다음과 같습니다.

           0. 아래의 과정들을 진행했는데도 오류가 난다면, 데이터 안에 값 자체를 전처리해주어야 합니다.
                null,nan과 같은 결측값을 대체해주거나, 문자나 리스트와 같은 값들로 변경해주어야 합니다.
                df['genre'] = df['genre'].apply(lambda x: x.split("|"))
                df['genre'] = df['genre'].apply(lambda x: rm_whitespace(x))
                df['actors'] = df['actors'].fillna('없음')
           1. 데이터 전처리 ~ 유저 평가하기 이전 까지의 코드 중에 수정할 곳들이 존재합니다
           2. "# n단계 조건 : 새로운 변수 추가" 라는 주석이 존재하는 곳에 갑니다
           3. "# new_feature = 'genre_people_country'#<<여기에 해당 형식으로 추가" 아래의 내용을 참고해서 코드를 완료합니다

# 데이터 불러오기

In [4]:
# 데이터가 커서 길이를 설정해놓았습니다. nrows=10000 자체를 없애거나 숫자를 변경해서 데이터 크기를 조정해보세요.
df = pd.read_csv('test2.csv', nrows = 100000)
df_stopwords = pd.read_csv('data/한국어불용어100.txt',sep='\t', header=None,names=['words','b','c'])
lst_words = df_stopwords['words'].values

In [9]:
df.columns = ['userr','iitem1','ttime',
             'title','subtitle','main_genre','genre',
             'keyword','actors','country','price','summary']

# 데이터 전처리

In [10]:
df['summary'] = df['summary'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df['summary'] = df['summary'].apply(lambda x: preprocessing(x, lst_words))

# 1회 이상 시청 고객
df_user_watch=pd.DataFrame(df['userr'].value_counts())
df_user_watch.columns=['cnt_watch']
#######################################################################################################
# 학습데이터를 몇 편이상을 시청한 user로쓸지
lst_user_real = df_user_watch.loc[df_user_watch['cnt_watch']>=10].index
#######################################################################################################

df = df.loc[df['userr'].isin(lst_user_real)]
# 유저 아이디 인코딩
le = LabelEncoder()
df['userr'] = le.fit_transform(df['userr']).astype(int)

df['genre'] = df['genre'].apply(lambda x: x.split("|"))
df['genre'] = df['genre'].apply(lambda x: rm_whitespace(x))
df['actors'] = df['actors'].fillna('없음')
df['country'] = df['country'].fillna('없음')
df['genre'] = df['genre'].fillna('없음')
df['year']=df['ttime'].apply(lambda x: str(x)[:4])
df['month']=df['ttime'].apply(lambda x: str(x)[4:6])
df['day']=df['ttime'].apply(lambda x: str(x)[6:8])
df['hour']=df['ttime'].apply(lambda x: str(x)[8:10])
df['min']=df['ttime'].apply(lambda x: str(x)[10:12])
df['sec']=df['ttime'].apply(lambda x: str(x)[12:])

AttributeError: 'list' object has no attribute 'split'

In [11]:
# 전체 데이터셋의 user, item 수 확인
user_list = list(df['userr'].unique())
item_list = list(df['iitem1'].unique())
num_users = len(user_list)
num_items = len(item_list)

# train, test set 나누기
#######################################################################################################
#train-test 나누기 
#랜덤버전
train, test = train_test_split(df, test_size=0.2, stratify=df['userr'],random_state = 1234)
#시간 선택 버전
#flag_range = int(df.shape[0]*80)
#df= df.sort_values(by=['year','month','day','hour','min','sec'],ascending=True)
#train = df[:flag_range]
#test = df[flag_range:]
#######################################################################################################

# 전체 데이터셋을 돌면서 모든 종류의 영화 장르, 국가, 배우 확인


# 1단계 조건 : 새로운 변수 추가
selected_features = ["genre", "country", "actors", "summary"] #<< 여기에 추가할 컬럼명 추가
all_genre_list = []
all_country_list = []
all_people_list = []
all_plot_list = [] 
# all_new_column_list = [] #<< 여기에 추가할 컬럼명으로 리스트 생성
for index, row in train.iterrows():
    genres = row["genre"]
    coutries = row["country"]
    people = row["actors"]
    plots = row["summary"]
    # new_columns = row['columns'] #<<여기에 해당 형식으로 추가
    #for genre in genres:
    #    if genre not in all_genre_list:
    #        all_genre_list.append(genre) #<<여기에 해당 형식으로 추가
    for genre in genres:
        if genre not in all_genre_list:
            all_genre_list.append(genre)
    for country in coutries:
        if country not in all_country_list:
            all_country_list.append(country)
    for person in people:
        if person not in all_people_list:
            all_people_list.append(person)
    for plot in plots:
        if plot not in all_plot_list:
            all_plot_list.append(plot)
num_genres = len(all_genre_list)
num_countries = len(all_country_list)
num_people = len(all_people_list)
num_plot = len(all_plot_list)
# num_column = len(all_new_columns_list) #<<여기에 해당 형식으로 추가

# one-hot encoding
def binary(feature_list, all_feature_list):
    binary_list = []
    for feature in all_feature_list:
        if feature in feature_list:
            binary_list.append(1)
        else:
            binary_list.append(0)
    
    return binary_list

# 2단계 조건 : 새로운 변수 추가
train['genre_bin'] = train['genre'].apply(lambda x: binary(x, all_genre_list))
train['country_bin'] = train['country'].apply(lambda x: binary(x, all_country_list))
train['people_bin'] = train['actors'].apply(lambda x: binary(x, all_people_list))
train['plot_bin'] = train['summary'].apply(lambda x: binary(x, all_plot_list))
#train['columns_bin'] = train['columns'].apply(lambda x: binary(x, all_columns_list)) #<<여기에 해당 형식으로 추가

In [12]:
# 3단계 조건 : 새로운 변수 추가.
new_feature = 'genre_plot'
# new_feature = 'genre_people_country'#<<여기에 해당 형식으로 추가
train[new_feature] = train['genre_bin'] + train['plot_bin']
# train[new_feature] = train['genre_bin'] + train['people_bin'] + train['country_bin']#<<여기에 해당 형식으로 추가

## 특정 유저에 대해서 평가하기

In [13]:
grouped_sum = train[new_feature].groupby(by=train['userr']).sum()
num_features = len(train[new_feature].iloc[0])

user_bin = {}
for user_idx in user_list:
    total_bin = np.zeros(num_features)
    num_dim = int(len(grouped_sum[user_idx])/num_features)

    for i in range(num_dim):
        one_movie = np.array(grouped_sum[user_idx][i*num_features:(i+1)*num_features])
        zipped_lists = zip(total_bin, one_movie)
        total_bin = [x + y for (x, y) in zipped_lists]

    total_bin = np.array(total_bin)
    user_bin[user_idx] = (total_bin, num_dim)

In [14]:
# 특정 user의 one-hot vector 확인해보기
user_id = 10
total_bin = user_bin[user_id][0]
num_movies = user_bin[user_id][1]

# combined one-hot vector를 가지고 다른 item들과의 cosine similarity 계산
norm_bin = total_bin / num_movies

# unique item 추리기
unique_items = train[['iitem1', 'title', 'genre', 'genre_bin', 'country', new_feature]].drop_duplicates(['iitem1'])

#############################################################################
#테스트할때 이전에 시청한 콘텐츠는 제외하고 평가하는것과 포함하여 평가하는것
# 특정 user가 본 영화들 제외
train_items_by_user = train.loc[train.userr==user_id]
unique_items = unique_items[~unique_items['iitem1'].isin(train_items_by_user['iitem1'])]
#unique_items = unique_items
#############################################################################
unique_items['similarity'] = unique_items[new_feature].apply(lambda x: np.array(x).dot(norm_bin) / (np.array(x).sum() + 1e-10))
unique_items.head()

# cosine similarity를 토대로 top-k item 구하기
sorted_items = unique_items.sort_values(by=['similarity'], axis=0, ascending=False)
sorted_items.head()

top_k = 10
top_k_items = list(sorted_items['iitem1'][:top_k])
top_item_df = sorted_items[['iitem1', 'title', 'genre']].drop_duplicates(['iitem1'])
# 예측한 top-k items
top_item_df[top_item_df['iitem1'].isin(top_k_items[:top_k])]

# user가 실제로 본 영화들
user_id = 10
test_items_by_user = test.loc[test.userr==user_id]
test_items_by_user[['userr', 'iitem1', 'title', 'genre']]

# user 한 명에 대한 평가
top_k = 200
pred_u = list(sorted_items['iitem1'])
target_u = list(test_items_by_user['iitem1'])

prec, recall, ndcg = compute_metrics(pred_u, target_u, top_k)
print(f"Precison@{top_k}: {prec:.4f}")
print(f"Recall@{top_k}: {recall:.4f}")
print(f"NDCG@{top_k}: {ndcg:.4f}")

Precison@200: 0.0000
Recall@200: 0.0000
NDCG@200: 0.0000


## 전체 유저에 대해서 평가하기

In [15]:
# 전체 user에 대한 평가
top_k = 200
prec_list = []
recall_list = []
ndcg_list = []

# unique item 추리기
ori_unique_items = train[['iitem1', 'title', 'genre', 'genre_bin', 'country', new_feature]].drop_duplicates(['iitem1'])

for user_id in user_list:
    total_bin = user_bin[user_id][0]
    num_movies = user_bin[user_id][1]

    # combined one-hot vector를 가지고 다른 item들과의 cosine similarity 계산
    norm_bin = total_bin / num_movies

    #############################################################################
    #테스트할때 이전에 시청한 콘텐츠는 제외하고 평가하는것과 포함하여 평가하는것
    # 특정 user가 본 영화들 제외
    train_items_by_user = train.loc[train.userr==user_id]
    unique_items = unique_items[~unique_items['iitem1'].isin(train_items_by_user['iitem1'])]
    #unique_items = unique_items
    #############################################################################
    unique_items['similarity'] = unique_items[new_feature].apply(lambda x: np.array(x).dot(norm_bin) / (np.array(x).sum() + 1e-10))

    # cosine similarity를 토대로 top-k item 구하기
    sorted_items = unique_items.sort_values(by=['similarity'], axis=0, ascending=False)

    test_items_by_user = test.loc[test.userr==user_id]
    pred_u = list(sorted_items['iitem1'])
    target_u = list(test_items_by_user['iitem1'])

    prec, recall, ndcg = compute_metrics(pred_u, target_u, top_k)
    prec_list.append(prec)
    recall_list.append(recall)
    ndcg_list.append(ndcg)

print(f"Precision@{top_k}: {np.mean(prec_list):.4f}")
print(f"Recall@{top_k}: {np.mean(recall_list):.4f}")
print(f"NDCG@{top_k}: {np.mean(ndcg_list):.4f}")

Precision@200: 0.0001
Recall@200: 0.0082
NDCG@200: 0.0029
